# Полезные графики

In [1]:
import pandas as pd
import plotly
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import plotly.express as px
import plotly.graph_objects as go

In [3]:
data = pd.read_excel('tiny_data.xlsx')
data.shape

(114482, 16)

In [64]:
data['date'] = pd.to_datetime(data['date'])
hol = data[(data['date'] > pd.to_datetime('2019-12-20')) & (data['date'] < pd.to_datetime('2020-01-08'))]
hol.drop(columns=['customerId', 'inn'], inplace=True)

C:\Users\zemer\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [67]:
count_brand = pd.DataFrame(hol.brand.value_counts())
count_brand.columns=['cnt']
count_brand.index.rename('brand', inplace=True)
count_brand.head()
hol = pd.merge(hol, count_brand, how='left', on='brand')

In [131]:
fig = go.Figure()

brands = hol[hol.cnt > 30].brand
vis = data[data.brand.apply(lambda x: x in set(brands))].brand.sample(hol.shape[0])

fig.add_trace(go.Histogram(x=vis))
fig.add_trace(go.Histogram(x=hol[hol.brand.apply(lambda x: x in set(brands))].brand)) #nbinsx=900))
              
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.6)

fig.show()

In [111]:
hol[hol.brand == 'oral-b'].head(2)

,checkId,date,product,brand,tag,price,amount,sum,cashback,company,count_check,count_product,count_brand,cnt
38,00282ad38fb8fb8ae1e4b7167cb71010,2019-12-23 10:35:00,oral-b vitality 3d white luxe з/щ электрич(прокт,oral-b,forp&g; электрическая; уход за полостью рта; з...,999.0,1.0,999.0,99.9,"0П ""ЛФ"" ЗАО ""ТАНДЕР-10"" ЗАО ""ТАНДЕР""",14,35,18,195
100,007ceed87bc7ca74eb509e3e4207f94f,2019-12-30 12:10:00,oral-b з/щ kids (5-7 лет) дет.,oral-b,красота и здоровье; уход за полостью рта; зубн...,44.9,2.0,89.8,0.0,Неизвестно,1,4,3,195


In [166]:
df = hol[hol.brand == 'oral-b']
#df['company'] = df.index
df['company'] = df['company'].apply(lambda x: x[:45]) 

fig = px.pie(df, values='sum', names='company',
             title='Суммарная стоимость покупок бренда oral-b в разных организациях на праздники',
             hover_data=['sum'])

fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()

C:\Users\zemer\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [158]:
df = data[data.brand == 'oral-b']
#df['company'] = df.index
df['company'] = df['company'].apply(lambda x: x[:45]) 

fig = px.pie(df, values='sum', names='company',
             title='Суммарная стоимость покупок бренда oral-b в разных организациях в обычные дни',
             hover_data=['sum'])

fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()

C:\Users\zemer\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [129]:
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x=data[data.brand == 'oral-b'].price,
        histnorm='percent',
        nbinsx=40,
        name='general'
    )
)

fig.add_trace(
    go.Histogram(
        x=hol[hol.brand == 'oral-b'].price, 
        histnorm='percent',
        nbinsx=40,
        name='holidays'
    )
)
fig.update_layout(title_text='Распределение стоимости покупок бренда oral-b', bargap=0.2, barmode='overlay')
fig.update_traces(opacity=0.6)

fig.show()

print(
    'Средняя стоимость покупки бренда oral-b: {} в праздники, {} обычно'.format(
    round(hol[hol.brand == 'oral-b'].price.mean(), 3), 
    round(data[data.brand == 'oral-b'].price.mean(), 3)
    )
)

Средняя стоимость покупки бренда oral-b: 1876.429 в праздники, 1440.192 обычно


In [155]:
fig = go.Figure()
hdf = hol.groupby('checkId').sum()
hdf = hdf[hdf['sum'] < 10000]

df = data.groupby('checkId').sum()
df = df[df['sum'] < 10000].sample(hdf.shape[0])

fig.add_trace(
    go.Histogram(
        x=df['sum'],
        nbinsx=60,
        name='general'
    )
)

fig.add_trace(
    go.Histogram(
        x=hdf['sum'],
        nbinsx=60,
        name='holidays'
    )
)
fig.update_layout(title_text='Распределение размера чека', bargap=0.2, barmode='overlay')
fig.update_traces(opacity=0.6)

fig.show()

In [163]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=hol[hol.price > 500].brand))
              
fig.update_layout(title_text='Распредедение брендов для покупок свяше 500р')

fig.show()

In [172]:
df = hol[hol['brand'] == 'oral-b'].groupby('company').mean()
df['company'] = df.index
fig = px.bar(df, x='company', y='price')
fig.update_layout(title_text='Средняя стоимость покупки по магазинам, в которых покупали oral-b')
fig.show()

In [175]:
df = data[(data.brand == 'oral-b') & (data['price'] > 700)]
#df['company'] = df.index
df['company'] = df['company'].apply(lambda x: x[:45]) 

fig = px.pie(df, values='sum', names='company',
             title='Где обычно покупают дорогие товары oral-b',
             hover_data=['sum'])

fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()

C:\Users\zemer\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [196]:
fig = go.Figure()
df = pd.DataFrame(data[data['brand'] == 'oral-b']['product'].value_counts())
df['name'] = df.index
df = df[df['product'] > 3]

fig = px.bar(df, x='name', y='product')
fig.update_layout(title_text='Модели проданных щеток oral-b')

fig.show()